<h1> SourceDataModel 

<h3> in database in tabel Sales_salesterritory attributt 'Group' veranderd naar 'Group1' vanwege error in code`

In [1]:
import pandas as pd
import sqlite3
import warnings
import pyodbc
import numpy as np
import os

warnings.simplefilter('ignore')
print(pyodbc.drivers())

['ODBC Driver 18 for SQL Server', 'ODBC Driver 17 for SQL Server']


<h3> inlezen database: AdventureWorks

In [2]:
# Path to the folder containing the CSV files
csv_folder = "databases/AdventureWorks"

# List all CSV files in the folder
csv_files = [f for f in os.listdir(csv_folder) if f.endswith('.csv')]
print(f"Found CSV files: {csv_files}")

# Initialize a dictionary to store DataFrames
df_AdventureWorks = {}

# Loop through each CSV file and load it into a DataFrame
for csv_file in csv_files:
    file_path = os.path.join(csv_folder, csv_file)
    print(f"Loading file: {file_path}")
    try:
        # Try reading the file with a fallback encoding
        df = pd.read_csv(file_path, encoding='latin1')  # Use 'latin1' or 'iso-8859-1' if UTF-8 fails
        # Store the DataFrame in the dictionary with the table name as the key
        table_name = os.path.splitext(csv_file)[0]
        df_AdventureWorks[table_name] = df
    except Exception as e:
        print(f"Failed to load {csv_file}: {e}")

# Access individual DataFrames by their table name
for table_name, df in df_AdventureWorks.items():
    print(f"Table: {table_name}, Rows: {len(df)}\n{df.head()}\n")
    print("Columns:", df.columns.tolist())


#function to clean nan values
def clean_nan_values(dw):
    for table_name, df in dw.items():
        # Zet alle NaN naar None zodat SQL Server NULL kan verwerken
        dw[table_name] = df.astype(object).where(pd.notnull(df), None)
    return dw

# Clean NaN values
df_AdventureWorks = clean_nan_values(df_AdventureWorks)




Found CSV files: ['Sales_Store.csv', 'Production_BillOfMaterials.csv', 'Sales_SalesTerritory.csv', 'Person_Address.csv', 'Purchasing_Vendor.csv', 'Person_Person.csv', 'Purchasing_PurchaseOrderHeader.csv', 'Sales_SalesOrderDetail.csv', 'HumanResources_Employee.csv', 'Production_ProductCategory.csv', 'Production_Product.csv', 'HumanResources_Department.csv', 'Sales_SalesOrderHeader.csv', 'Purchasing_PurchaseOrderDetail.csv', 'Sales_Customer.csv']
Loading file: databases/AdventureWorks/Sales_Store.csv
Loading file: databases/AdventureWorks/Production_BillOfMaterials.csv
Loading file: databases/AdventureWorks/Sales_SalesTerritory.csv
Loading file: databases/AdventureWorks/Person_Address.csv
Loading file: databases/AdventureWorks/Purchasing_Vendor.csv
Loading file: databases/AdventureWorks/Person_Person.csv
Loading file: databases/AdventureWorks/Purchasing_PurchaseOrderHeader.csv
Loading file: databases/AdventureWorks/Sales_SalesOrderDetail.csv
Loading file: databases/AdventureWorks/HumanRe

<h3> inlezen database: aenc.sqlite

In [3]:
# Path to the SQLite database
sqlite_file = "databases/aenc.sqlite"

# Connect to the SQLite database
conn = sqlite3.connect(sqlite_file)

# Get a list of all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(query, conn)
table_names = tables['name'].tolist()
print(f"Found tables: {table_names}")

# Initialize a dictionary to store DataFrames
df_aenc = {}

# Loop through each table and load it into a DataFrame
for table_name in table_names:
    print(f"Loading table: {table_name}")
    try:
        # Read the table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table_name};", conn)
        # Store the DataFrame in the dictionary with the table name as the key
        df_aenc[table_name] = df
    except Exception as e:
        print(f"Failed to load table {table_name}: {e}")

# Close the database connection
conn.close()

# Access individual DataFrames by their table name
for table_name, df in df_aenc.items():
    print(f"Table: {table_name}, Rows: {len(df)}\n{df.head()}\n")


# Clean NaN values in the DataFrames
df_aenc = clean_nan_values(df_aenc)

Found tables: ['Department', 'Product', 'State', 'Bonus', 'Customer', 'Employee', 'Sales_Order', 'Sales_Order_Item']
Loading table: Department
Loading table: Product
Loading table: State
Loading table: Bonus
Loading table: Customer
Loading table: Employee
Loading table: Sales_Order
Loading table: Sales_Order_Item
Table: Department, Rows: 5
   dept_id  dept_name  dept_head_id
0      100      R & D           501
1      200      Sales           902
2      300    Finance          1293
3      400  Marketing          1576
4      500   Shipping           703

Table: Product, Rows: 10
    id          name description          prod_size   color  quantity  \
0  300     Tee Shirt    Tank Top              Small   White        18   
1  301     Tee Shirt      V-neck             Medium  Orange        39   
2  302     Tee Shirt   Crew Neck  One size fits all   Black        72   
3  400  Baseball Cap  Cotton Cap  One size fits all   Black        92   
4  401  Baseball Cap    Wool cap  One size fits all

<h3> Inlezen database: NorthWind SQL server

In [4]:
# Verbindingsgegevens
server = '127.0.0.1'        
port = '1433'               
database = 'NorthWind'         
username = 'SA'             
password = 'iDTyjZx7dRL4'  

# Connection string
connection_string = (
    f"DRIVER={{ODBC Driver 17 for SQL Server}};"
    f"SERVER={server},{port};"
    f"DATABASE={database};"
    f"UID={username};"
    f"PWD={password};"
    "TrustServerCertificate=yes;"
    "Timeout=30;"
)

# Maak verbinding met de database
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

# Haal alle tabellen op
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE = 'BASE TABLE'")
tables = [row.TABLE_NAME for row in cursor.fetchall()]

# Dictionary om alle dataframes op te slaan
df_NorthWind = {}

# Loop door alle tabellen en laad ze in Pandas DataFrames
for table in tables:
    query = f"SELECT * FROM [{table}]"
    df = pd.read_sql(query, conn)
    df_NorthWind[table] = df
    print(f"Tabel '{table}' ingelezen met {df.shape[0]} rijen en {df.shape[1]} kolommen.")

# Sluit de verbinding
conn.close()

# Print de kolomnamen en de eerste paar rijen van elke DataFrame
for table_name, df in df_NorthWind.items():
    print(f"\nTable: {table_name}")
    print("Columns:", df.columns.tolist())
    print(df.head())
    




Tabel 'Employees' ingelezen met 9 rijen en 18 kolommen.
Tabel 'Categories' ingelezen met 8 rijen en 4 kolommen.
Tabel 'Customers' ingelezen met 91 rijen en 11 kolommen.
Tabel 'Shippers' ingelezen met 3 rijen en 3 kolommen.
Tabel 'Suppliers' ingelezen met 29 rijen en 12 kolommen.
Tabel 'Orders' ingelezen met 830 rijen en 14 kolommen.
Tabel 'Products' ingelezen met 77 rijen en 10 kolommen.
Tabel 'Order Details' ingelezen met 2155 rijen en 5 kolommen.
Tabel 'CustomerCustomerDemo' ingelezen met 0 rijen en 2 kolommen.
Tabel 'CustomerDemographics' ingelezen met 0 rijen en 2 kolommen.
Tabel 'Region' ingelezen met 4 rijen en 2 kolommen.
Tabel 'Territories' ingelezen met 53 rijen en 3 kolommen.
Tabel 'EmployeeTerritories' ingelezen met 49 rijen en 2 kolommen.

Table: Employees
Columns: ['EmployeeID', 'LastName', 'FirstName', 'Title', 'TitleOfCourtesy', 'BirthDate', 'HireDate', 'Address', 'City', 'Region', 'PostalCode', 'Country', 'HomePhone', 'Extension', 'Photo', 'Notes', 'ReportsTo', 'PhotoPa

<h3>maak dictionary aan voor alle goede dataframes


In [5]:
dfs_sourcedatamodel = {}

<h3> table: Production_ProductCategory
<h4> bronnen: Production_ProductCategory + Categories

In [6]:
# Data opnieuw laden om dubbele kolommen te voorkomen
df_product_category = df_AdventureWorks.get("Production_ProductCategory").copy()
df_categories = df_NorthWind.get("Categories").copy()

# Voeg 'Source' kolom toe
df_product_category['Source'] = 'AdventureWorks'
df_categories['Source'] = 'NorthWind'


# Voeg een unieke sleutel toe (MergedCategoryID) als deze nog niet bestaat
if "MergedCategoryID" not in df_product_category.columns:
    df_product_category.insert(0, "MergedCategoryID", range(1, len(df_product_category) + 1))

if "MergedCategoryID" not in df_categories.columns:
    df_categories.insert(0, "MergedCategoryID", range(len(df_product_category) + 1, len(df_product_category) + len(df_categories) + 1))

# Hernoem kolommen zodat ze overeenkomen
df_categories.rename(columns={
    "CategoryID": "ProductCategoryID", 
    "CategoryName": "Name"
}, inplace=True)

# Combineer de twee tabellen onder elkaar
merged_df_productCategorie = pd.concat([df_product_category, df_categories], ignore_index=True)

# Vul NaN-waarden in met lege strings (optioneel)
merged_df_productCategorie.fillna("", inplace=True)


#mapping van de categorieen per bron
category_mapping = merged_df_productCategorie[['ProductCategoryID', 'Source', 'MergedCategoryID']]



# Print het resultaat
print(merged_df_productCategorie)

    MergedCategoryID ProductCategoryID            Name  \
0                  1                 1           Bikes   
1                  2                 2      Components   
2                  3                 3        Clothing   
3                  4                 4     Accessories   
4                  5                 1       Beverages   
5                  6                 2      Condiments   
6                  7                 3     Confections   
7                  8                 4  Dairy Products   
8                  9                 5  Grains/Cereals   
9                 10                 6    Meat/Poultry   
10                11                 7         Produce   
11                12                 8         Seafood   

                                  rowguid       ModifiedDate          Source  \
0   {CFBDA25C-DF71-47A7-B81B64EE161AA37C}  04/30/18 00:00:00  AdventureWorks   
1   {C657828D-D808-4ABA-91A3AF2CE02300E9}  04/30/18 00:00:00  AdventureWorks   
2   {

<h3> table: Production_Product
<h4> bronnen: Production_Product + Products + Product

In [7]:

# Data opnieuw laden om dubbele kolommen te voorkomen
df_production_product = df_AdventureWorks.get("Production_Product").copy()
df_products = df_NorthWind.get("Products").copy()
df_product = df_aenc.get("Product").copy()

# Voeg 'Source' kolom toe
df_production_product['Source'] = 'AdventureWorks'
df_products['Source'] = 'NorthWind'
df_product['Source'] = 'aenc'

# Maak een unieke MergedID voor elke rij
df_production_product.insert(0, "MergedID", range(1, len(df_production_product) + 1))
df_products.insert(0, "MergedID", range(len(df_production_product) + 1, len(df_production_product) + len(df_products) + 1))
df_product.insert(0, "MergedID", range(len(df_production_product) + len(df_products) + 1, 
                                       len(df_production_product) + len(df_products) + len(df_product) + 1))

# Zorg dat alle tabellen dezelfde kolommen hebben
df_products.rename(columns={"ProductName": "Name", "CategoryID": "ProductCategoryID", "UnitPrice": "ListPrice"}, inplace=True)
df_product.rename(columns={"id": "ProductID", "name": "Name", "prod_size": "Size", "unit_price": "ListPrice", "Category": "ProductCategoryID"}, inplace=True)

# Voeg ontbrekende kolommen toe met lege waarden
all_columns = set(df_production_product.columns).union(set(df_products.columns)).union(set(df_product.columns))
for df in [df_production_product, df_products, df_product]:
    for col in all_columns:
        if col not in df.columns:
            df[col] = None

# Zet alle tabellen onder elkaar
merged_df_product = pd.concat([df_production_product, df_products, df_product], ignore_index=True)

# Vul NaN-waarden in met lege strings (optioneel)
merged_df_product.fillna("", inplace=True)

# Verwijder 'picture_name' en 'DiscontinuedDate' kolommen
merged_df_product.drop(columns=["picture_name"], inplace=True, errors="ignore")

# Hernoem 'ReorderLevel' naar 'ReorderPoint' en zet ze samen in één kolom
if "ReorderLevel" in merged_df_product.columns and "ReorderPoint" in merged_df_product.columns:
    merged_df_product["ReorderPoint"] = merged_df_product["ReorderPoint"].combine_first(merged_df_product["ReorderLevel"])
    merged_df_product.drop(columns=["ReorderLevel"], inplace=True)

# Hernoem 'color' zodat beide versies samengevoegd worden in één kolom
if "color" in merged_df_product.columns and "Color" in merged_df_product.columns:
    merged_df_product["Color"] = merged_df_product["Color"].combine_first(merged_df_product["color"])
    merged_df_product.drop(columns=["color"], inplace=True)

product_mapping = merged_df_product[['ProductID', 'Source', 'MergedID']]

# Merge op ProductCategoryID + Source om conflicten te vermijden
merged_df_product = merged_df_product.merge(
    category_mapping,
    how='left',
    left_on=['ProductCategoryID', 'Source'],
    right_on=['ProductCategoryID', 'Source']
)

# Vervang de oude ProductCategoryID door de nieuwe MergedCategoryID
merged_df_product['ProductCategoryID'] = merged_df_product['MergedCategoryID']
merged_df_product.drop(columns=['MergedCategoryID'], inplace=True, errors='ignore')




# Print het resultaat
print(merged_df_product)



     MergedID ProductID                   Name ProductNumber MakeFlag  \
0           1         1        Adjustable Race       AR-5381        0   
1           2         2           Bearing Ball       BA-8327        0   
2           3         3        BB Ball Bearing       BE-2349       -1   
3           4         4  Headset Ball Bearings       BE-2908        0   
4           5       316                  Blade       BL-2036       -1   
..        ...       ...                    ...           ...      ...   
586       587       500                  Visor                          
587       588       501                  Visor                          
588       589       600             Sweatshirt                          
589       590       601             Sweatshirt                          
590       591       700                 Shorts                          

    FinishedGoodsFlag Color SafetyStockLevel ReorderPoint StandardCost  ...  \
0                   0                   1000

<h3> table: Purchasing_Vendor

In [8]:

#get dataframe from dictionary
purchasing_vendor_df = df_AdventureWorks.get("Purchasing_Vendor")

# Check if the DataFrame exists
if purchasing_vendor_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Purchasing_Vendor"] = purchasing_vendor_df

    print(f"Rows: {len(purchasing_vendor_df)}")
    print(purchasing_vendor_df.head())
else:
    print("The table 'Purchasing_Vendor' does not exist in database.")

Rows: 104
  BusinessEntityID AccountNumber                     Name CreditRating  \
0             1492  AUSTRALI0001  Australia Bike Retailer            1   
1             1494  ALLENSON0001          Allenson Cycles            2   
2             1496  ADVANCED0001        Advanced Bicycles            1   
3             1498    TRIKES0001             Trikes, Inc.            2   
4             1500   MORGANB0001  Morgan Bike Accessories            1   

  PreferredVendorStatus ActiveFlag PurchasingWebServiceURL       ModifiedDate  
0                    -1         -1                    None  12/23/21 00:00:00  
1                    -1         -1                    None  04/25/21 00:00:00  
2                    -1         -1                    None  04/25/21 00:00:00  
3                    -1         -1                    None  02/03/22 00:00:00  
4                    -1         -1                    None  02/02/22 00:00:00  


<h3> table: sales_store

In [9]:

#get dataframe from dictionary
sales_store_df = df_AdventureWorks.get("Sales_Store")

# Check if the DataFrame exists
if sales_store_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Sales_Store"] = sales_store_df

    print(f"Rows: {len(sales_store_df)}")
    print(sales_store_df.head())
else:
    print("The table does not exist in database.")

Rows: 701
  BusinessEntityID                            Name SalesPersonID  \
0              292            Next-Door Bike Store           279   
1              294  Professional Sales and Service           276   
2              296                  Riders Company           277   
3              298              The Bike Mechanics           275   
4              300               Nationwide Supply           286   

                                 rowguid       ModifiedDate  
0  {A22517E3-848D-4EBE-B9D97437F3432304}  09/12/14 11:15:07  
1  {B50CA50B-C601-4A13-B07E2C63862D71B4}  09/12/14 11:15:07  
2  {337C3688-1339-4E1A-A08AB54B23566E49}  09/12/14 11:15:07  
3  {7894F278-F0C8-4D16-BD75213FDBF13023}  09/12/14 11:15:07  
4  {C3FC9705-A8C4-4F3A-9550EB2FA4B7B64D}  09/12/14 11:15:07  


<h3> tabel: employee_territories

In [10]:

#get dataframe from dictionary
employeeTerritories_df = df_NorthWind.get("EmployeeTerritories")

# Check if the DataFrame exists
if employeeTerritories_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["EmployeeTerritories"] = employeeTerritories_df

    print(f" Rows: {len(employeeTerritories_df)}")
    print(employeeTerritories_df.head())
else:
    print("The table does not exist in database.")

 Rows: 49
   EmployeeID TerritoryID
0           1       06897
1           1       19713
2           2       01581
3           2       01730
4           2       01833


<h3> table: Territories

In [11]:
#get dataframe from dictionary
territories_df = df_NorthWind.get("Territories")

# Check if the DataFrame exists
if employeeTerritories_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Territories"] = employeeTerritories_df

    print(f" Rows: {len(employeeTerritories_df)}")
    print(employeeTerritories_df.head())
else:
    print("The table does not exist in database.")

 Rows: 49
   EmployeeID TerritoryID
0           1       06897
1           1       19713
2           2       01581
3           2       01730
4           2       01833


<h3> table: Region

In [12]:
#get dataframe from dictionary
region_df = df_NorthWind.get("Region")

# Check if the DataFrame exists
if region_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Region"] = region_df

    print(f" Rows: {len(region_df)}")
    print(region_df.head())
else:
    print("The table does not exist in database.")

 Rows: 4
   RegionID                                  RegionDescription
0         1  Eastern                                       ...
1         2  Western                                       ...
2         3  Northern                                      ...
3         4  Southern                                      ...


<h3> Table: Bonus

In [13]:
#get dataframe from dictionary
bonus_df = df_aenc.get("Bonus")

# Check if the DataFrame exists
if bonus_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Bonus"] = bonus_df

    print(f" Rows: {len(bonus_df)}")
    print(bonus_df.head())
else:
    print("The table does not exist in database.")

 Rows: 37
  emp_id bonus_date bonus_amount
0    129  31-dec-24           50
1    195  31-dec-23           50
2    299  31-dec-23          200
3   1142  31-dec-23          100
4    129  31-dec-22           50


<h3> table: Sales_Customer

In [14]:
#get dataframe from dictionary
Sales_Customer_df = df_AdventureWorks.get("Sales_Customer")

# Check if the DataFrame exists
if Sales_Customer_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Sales_Customer"] = Sales_Customer_df

    print(f" Rows: {len(Sales_Customer_df)}")
    print(Sales_Customer_df.head())
else:
    print("The table does not exist in database.")

 Rows: 19820
  CustomerID PersonID StoreID TerritoryID AccountNumber  \
0          1     None   934.0           1    AW00000001   
1          2     None  1028.0           1    AW00000002   
2          3     None   642.0           4    AW00000003   
3          4     None   932.0           4    AW00000004   
4          5     None  1026.0           4    AW00000005   

                                 rowguid       ModifiedDate  
0  {3F5AE95E-B87D-4AED-95B4C3797AFCB74F}  09/12/24 11:15:07  
1  {E552F657-A9AF-4A7D-A645C429D6E02491}  09/12/24 11:15:07  
2  {130774B1-DB21-4EF3-98C8C104BCD6ED6D}  09/12/24 11:15:07  
3  {FF862851-1DAA-4044-BE7C3E85583C054D}  09/12/24 11:15:07  
4  {83905BDC-6F5E-4F71-B162C98DA069F38A}  09/12/24 11:15:07  


<h3> table: Sales_SalesTerritory

In [15]:
#get dataframe from dictionary
Sales_SalesTerritory_df = df_AdventureWorks.get("Sales_SalesTerritory")

# Check if the DataFrame exists
if Sales_SalesTerritory_df is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Sales_SalesTerritory"] = Sales_SalesTerritory_df

    print(f" Rows: {len(Sales_SalesTerritory_df)}")
    print(Sales_SalesTerritory_df.head())
else:
    print("The table does not exist in database.")

 Rows: 10
  TerritoryID       Name CountryRegionCode          Group       SalesYTD  \
0           1  Northwest                US  North America   7887186.7882   
1           2  Northeast                US  North America   2402176.8476   
2           3    Central                US  North America    3072175.118   
3           4  Southwest                US  North America  10510853.8739   
4           5  Southeast                US  North America   2538667.2515   

  SalesLastYear CostYTD CostLastYear                                rowguid  \
0  3298694.4938     0.0          0.0  {43689A10-E30B-497F-B0DE11DE20267FF7}   
1  3607148.9371     0.0          0.0  {00FB7309-96CC-49E2-83630A1BA72486F2}   
2  3205014.0767     0.0          0.0  {DF6E7FD8-1A8D-468C-B103ED8ADDB452C1}   
3  5366575.7098     0.0          0.0  {DC3E9EA0-7950-4431-942899DBCBC33865}   
4  3925071.4318     0.0          0.0  {6DC4165A-5E4C-42D2-809D4344E0AC75E7}   

        ModifiedDate  
0  04/30/08 00:00:00  
1  04/30/08 

<h3> table: Purchasing_PurchaseOrderHeader


In [16]:


#get dataframe from dictionary
Purchasing_PurchaseOrderHeader = df_AdventureWorks.get("Purchasing_PurchaseOrderHeader")

# Check if the DataFrame exists
if Purchasing_PurchaseOrderHeader is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Purchasing_PurchaseOrderHeader"] = Purchasing_PurchaseOrderHeader

    print(f" Rows: {len(Purchasing_PurchaseOrderHeader)}")
    print(Purchasing_PurchaseOrderHeader.head())
else:
    print("The table does not exist in database.")

 Rows: 4012
  PurchaseOrderID RevisionNumber Status EmployeeID VendorID ShipMethodID  \
0               1              4      4        258     1580            3   
1               2              4      1        254     1496            5   
2               3              4      4        257     1494            2   
3               4              4      3        261     1650            5   
4               5              4      4        251     1654            4   

           OrderDate           ShipDate  SubTotal    TaxAmt   Freight  \
0  04/16/21 00:00:00  04/25/21 00:00:00    201.04   16.0832     5.026   
1  04/16/21 00:00:00  04/25/21 00:00:00  272.1015   21.7681    6.8025   
2  04/16/21 00:00:00  04/25/21 00:00:00    8847.3   707.784  221.1825   
3  04/16/21 00:00:00  04/25/21 00:00:00  171.0765   13.6861    4.2769   
4  04/30/21 00:00:00  05/09/21 00:00:00   20397.3  1631.784  509.9325   

     TotalDue       ModifiedDate  
0    222.1492  04/25/21 00:00:00  
1    300.6721  04/25/2

<h3> table: Purchasing_PurchaseOrderDetail

In [ ]:
# Get the Purchasing_PurchaseOrderDetail DataFrame from the dictionary
Purchasing_PurchaseOrderDetail = df_AdventureWorks.get("Purchasing_PurchaseOrderDetail")

# Check if the DataFrame exists
if Purchasing_PurchaseOrderDetail is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Purchasing_PurchaseOrderDetail"] = Purchasing_PurchaseOrderDetail

    # Perform the merge to replace ProductID with MergedID
    Purchasing_PurchaseOrderDetail = Purchasing_PurchaseOrderDetail.merge(
        product_mapping,  # This contains ProductID, Source, and MergedID
        how='left',
        left_on=['ProductID'],  # Assuming ProductID exists in this table
        right_on=['ProductID']
    )

    # Replace ProductID with MergedID
    Purchasing_PurchaseOrderDetail['ProductID'] = Purchasing_PurchaseOrderDetail['MergedID']

    # Drop the MergedID column as it's no longer needed
    Purchasing_PurchaseOrderDetail.drop(columns=['MergedID'], inplace=True, errors='ignore')

    # Drop rows where Source is 'NorthWind'
    if 'Source' in Purchasing_PurchaseOrderDetail.columns:
        Purchasing_PurchaseOrderDetail = Purchasing_PurchaseOrderDetail[Purchasing_PurchaseOrderDetail['Source'] != 'NorthWind']
        Purchasing_PurchaseOrderDetail.reset_index(drop=True, inplace=True)
        print("Rows with Source == 'NorthWind' have been removed.")
    else:
        print("The 'Source' column does not exist in Purchasing_PurchaseOrderDetail.")

    # Update the dictionary with the modified DataFrame
    dfs_sourcedatamodel["Purchasing_PurchaseOrderDetail"] = Purchasing_PurchaseOrderDetail

    # Print the result
    print(f"Rows: {len(Purchasing_PurchaseOrderDetail)}")
    print(Purchasing_PurchaseOrderDetail.head())

   
    print("The updated table has been saved to 'Purchasing_PurchaseOrderDetail.csv'.")
else:
    print("The table 'Purchasing_PurchaseOrderDetail' does not exist in the database.")

Rows with Source == 'NorthWind' have been removed.
Rows: 8845
  PurchaseOrderID PurchaseOrderDetailID            DueDate OrderQty  \
0               1                     1  04/30/21 00:00:00        4   
1               2                     2  04/30/21 00:00:00        3   
2               2                     3  04/30/21 00:00:00        3   
3               3                     4  04/30/21 00:00:00      550   
4               4                     5  04/30/21 00:00:00        3   

   ProductID UnitPrice LineTotal ReceivedQty RejectedQty StockedQty  \
0          1     50.26    201.04         3.0         0.0        3.0   
1         38     45.12    135.36         3.0         0.0        3.0   
2         39   45.5805  136.7415         3.0         0.0        3.0   
3        203    16.086    8847.3       550.0         0.0      550.0   
4          4   57.0255  171.0765         2.0         1.0        1.0   

        ModifiedDate          Source  
0  04/23/21 00:00:00  AdventureWorks  
1  04/

<h3> table: Suppliers

In [18]:
#get dataframe from dictionary
Suppliers = df_NorthWind.get("Suppliers")

# Check if the DataFrame exists
if Suppliers is not None:
    # Add the DataFrame to the sourcedatamodel dictionary
    dfs_sourcedatamodel["Suppliers"] = Suppliers

    print(f" Rows: {len(Suppliers)}")
    print(Suppliers.head())
else:
    print("The table does not exist in database.")

 Rows: 29
   SupplierID                         CompanyName                 ContactName  \
0           1                      Exotic Liquids            Charlotte Cooper   
1           2          New Orleans Cajun Delights               Shelley Burke   
2           3           Grandma Kelly's Homestead               Regina Murphy   
3           4                       Tokyo Traders                Yoshi Nagase   
4           5  Cooperativa de Quesos 'Las Cabras'  Antonio del Valle Saavedra   

           ContactTitle                    Address         City    Region  \
0    Purchasing Manager             49 Gilbert St.       London      None   
1   Order Administrator             P.O. Box 78934  New Orleans        LA   
2  Sales Representative             707 Oxford Rd.    Ann Arbor        MI   
3     Marketing Manager  9-8 Sekimai Musashino-shi        Tokyo      None   
4  Export Administrator          Calle del Rosal 4       Oviedo  Asturias   

  PostalCode Country           Phone    